In [10]:
# === Library Import ===
import time
import os
import glob
import xlwings as xw
from pathlib import Path

In [26]:
# === Configuration ===
ROOT = Path.cwd() 
DATA_PATH = ROOT / "data"              # 相対パス（安全）
TARGET_SHEET = "October"           # 対象シート名（10月）

files = glob.glob(str(DATA_PATH / "*.xlsx")) 
file_names = [Path(f).name for f in files]
file_paths = [str(f) for f in files]    

print("実行ファイル：", ROOT)
print("参照フォルダ：", DATA_PATH)
print("参照シート：", TARGET_SHEET)
print("検出ファイル数：", len(file_names))
print("抽出ファイル一覧：", file_names)

実行ファイル： C:\Users\7102581\OneDrive - アサヒグループ\デスクトップ\Python\code\github用\01.店舗実績集約
参照フォルダ： C:\Users\7102581\OneDrive - アサヒグループ\デスクトップ\Python\code\github用\01.店舗実績集約\data
参照シート： October
検出ファイル数： 1
抽出ファイル一覧： ['銀座店.xlsx']


In [27]:
# === Data Extraction Function ===
def extract_store_data(filepath, sheet_name):
    """単一店舗のExcelファイルから必要情報を抽出"""
    wb = xw.Book(file_paths)
    ws = wb.sheets[TARGET_SHEET]

    place_code = ws.range(3, 3).value
    place_name = ws.range(4, 3).value
    budget = ws.range(8, 3).value
    pile = ws.range(18, 9).value

    records = []
    for row in range(24, 40):
        plan_no = ws.range(row, 5).value
        plan_name = ws.range(row, 6).value
        plan_class1 = ws.range(row, 7).value
        plan_class2 = ws.range(row, 8).value
        plan_budget = ws.range(row, 9).value
        plan_start = ws.range(row, 11).value
        plan_end = ws.range(row, 12).value

        records.append([
            place_code, place_name, budget, pile,
            plan_no, plan_name, plan_budget,
            plan_start, plan_end, plan_class1, plan_class2
        ])
    wb.close()
    return records


In [ ]:
# === Consolidate All Stores ===
start = time.time()
data = []

for path in file_paths:
    store_name = os.path.splitext(os.path.basename(path))[0]
    print(f"処理中: {store_name}")
    try:
        records = extract_store_data(path, TARGET_SHEET)
        data.extend(records)
    except Exception as e:
        print(f"⚠️ エラー発生: {store_name} - {e}")

print("\n✅ 全店舗処理完了")
print(f"総レコード数: {len(data)}")
print(f"処理時間: {time.time() - start:.2f} 秒")


In [ ]:
# === Optional: Export to Excel ===
import pandas as pd

columns = [
    "Code", "Name", "Budget", "Pile",
    "PlanNo", "PlanName", "PlanBudget",
    "Start", "End", "Class1", "Class2"
]
df = pd.DataFrame(data, columns=columns)
df.to_excel("./output/consolidated_sales.xlsx", index=False)

print("✅ 結果を 'output/consolidated_sales.xlsx' に出力しました")

In [28]:
# ==========================================
# Store Sales Excel Consolidation (Portfolio ver.)
# ==========================================
# - フォルダ内の店舗別Excelを走査し、指定シートから必要セルを抽出
# - file_names は表示用、file_paths は処理用として分離
# - 出力は ./output/consolidated_sales.xlsx
# ------------------------------------------

import time
from pathlib import Path
import xlwings as xw
import pandas as pd

# === Configuration ===
ROOT = Path.cwd()                 # このノート/スクリプトの場所
DATA_PATH = ROOT / "data"         # 店舗Excel置き場（相対パス推奨）
TARGET_SHEET = "October"          # 対象シート名（例：10月 → "10月"）

# === Prepare file lists ===
files = list(DATA_PATH.glob("*.xlsx")) + list(DATA_PATH.glob("*.xlsm"))  # 両対応
file_names = [f.name for f in files]      # 表示用（ファイル名）
file_paths = [str(f) for f in files]      # 処理用（フルパス文字列）

print("実行ディレクトリ：", ROOT)
print("参照フォルダ　　：", DATA_PATH)
print("参照シート　　　：", TARGET_SHEET)
print("検出ファイル数　：", len(file_names))
print("抽出ファイル一覧：", file_names)

# === Extraction function ===
def extract_store_data(filepath: str, sheet_name: str):
    """単一店舗Excelから必要情報を抽出して行リストを返す"""
    records = []
    wb = None
    try:
        wb = xw.Book(filepath)
        # シート存在チェック
        names = [s.name for s in wb.sheets]
        if sheet_name not in names:
            raise FileNotFoundError(f"シート '{sheet_name}' が見つかりません。候補: {names}")

        ws = wb.sheets[sheet_name]

        # ヘッダ系
        place_code = ws.range(3, 3).value   # C3
        place_name = ws.range(4, 3).value   # C4
        budget     = ws.range(8, 3).value   # C8
        pile       = ws.range(18, 9).value  # I18

        # 明細行（必要に応じて範囲を調整）
        for row in range(24, 40):
            plan_no     = ws.range(row, 5).value   # E
            plan_name   = ws.range(row, 6).value   # F
            plan_class1 = ws.range(row, 7).value   # G
            plan_class2 = ws.range(row, 8).value   # H
            plan_budget = ws.range(row, 9).value   # I
            plan_start  = ws.range(row, 11).value  # K
            plan_end    = ws.range(row, 12).value  # L

            records.append([
                place_code, place_name, budget, pile,
                plan_no, plan_name, plan_budget,
                plan_start, plan_end, plan_class1, plan_class2
            ])
    finally:
        if wb is not None:
            wb.close()
    return records

# === Consolidate all stores ===
start = time.time()
data = []

for path in file_paths:
    store_name = Path(path).stem
    print(f"処理中: {store_name}")
    try:
        records = extract_store_data(path, TARGET_SHEET)
        data.extend(records)
    except Exception as e:
        print(f"⚠️ エラー: {store_name} - {e}")

elapsed = time.time() - start
print("\n✅ 全店舗処理完了")
print(f"総レコード数: {len(data)}")
print(f"処理時間: {elapsed:.2f} 秒（約 {elapsed/60:.1f} 分）")

# === To DataFrame & Export ===
columns = [
    "Code", "Name", "Budget", "Pile",
    "PlanNo", "PlanName", "PlanBudget",
    "Start", "End", "Class1", "Class2"
]
df = pd.DataFrame(data, columns=columns)

OUTPUT_DIR = ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)
out_path = OUTPUT_DIR / "consolidated_sales.xlsx"
df.to_excel(out_path, index=False)
print(f"📦 出力: {out_path}")


実行ディレクトリ： C:\Users\7102581\OneDrive - アサヒグループ\デスクトップ\Python\code\github用\01.店舗実績集約
参照フォルダ　　： C:\Users\7102581\OneDrive - アサヒグループ\デスクトップ\Python\code\github用\01.店舗実績集約\data
参照シート　　　： October
検出ファイル数　： 1
抽出ファイル一覧： ['銀座店.xlsx']
処理中: 銀座店

✅ 全店舗処理完了
総レコード数: 16
処理時間: 6.52 秒（約 0.1 分）
📦 出力: C:\Users\7102581\OneDrive - アサヒグループ\デスクトップ\Python\code\github用\01.店舗実績集約\output\consolidated_sales.xlsx
